In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/curated/domain_final.csv")

In [3]:
df.head()


,Unnamed: 0,uri,name,cost_text,type,school_title,school_distance,suburb_sold,suburb_avg_day,suburb_population,...,dist_nearest_secondary_school,Beds,Bath,Parking,SA2_CODE21,avg_income,Median_tot_fam_inc_weekly,Tot_P_P,Median_age_persons,Median_rent_weekly
0,0,https://www.domain.com.au/1202-620-collins-str...,1202/620 Collins Street Melbourne VIC 3000,$500 per week,Apartment / Unit / Flat,University High School,2.4,596.0,147.0,47279.0,...,380.6,2.0,1.0,1,206041503,2035.430097,2248.0,9848.0,30.0,375.0
1,2,https://www.domain.com.au/5707b-639-little-lon...,5707B/639 Little Lonsdale Street Melbourne VIC...,$570,Apartment / Unit / Flat,University High School,1.9,387.0,120.0,47279.0,...,355.7,1.0,1.0,0,206041503,2035.430097,2248.0,9848.0,30.0,375.0
2,4,https://www.domain.com.au/1202-31-spring-stree...,1202/31 Spring Street Melbourne VIC 3000,$570.00,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4,596.0,147.0,47279.0,...,703.0,2.0,1.0,1,206041503,2035.430097,2248.0,9848.0,30.0,375.0
3,5,https://www.domain.com.au/623-539-st-kilda-roa...,623/539 St Kilda Road Melbourne VIC 3000,$475 per week,Apartment / Unit / Flat,South Yarra Primary School,0.7,387.0,120.0,47279.0,...,584.0,1.0,1.0,1,206041503,2035.430097,2248.0,9848.0,30.0,375.0
4,6,https://www.domain.com.au/315-118-russell-stre...,315/118 Russell Street Melbourne VIC 3000,$550 per week,Apartment / Unit / Flat,Carlton Gardens Primary School,1.3,387.0,120.0,47279.0,...,497.6,1.0,1.0,0,206041503,2035.430097,2248.0,9848.0,30.0,375.0


In [4]:
affortable_df = pd.DataFrame(df,columns=["postcode","cost","avg_income"])

In [5]:
affortable_df["cost_gap"]=affortable_df["cost"]/affortable_df["avg_income"]

In [6]:
affortable_df.head()


,postcode,cost,avg_income,cost_gap
0,3000,500.0,2035.430097,0.245648
1,3000,570.0,2035.430097,0.280039
2,3000,570.0,2035.430097,0.280039
3,3000,475.0,2035.430097,0.233366
4,3000,550.0,2035.430097,0.270213


In [7]:
affortable_df.shape

(10158, 4)

In [8]:
affortable_df.isnull().sum()

postcode        0
cost            0
avg_income    141
cost_gap      141
dtype: int64

In [9]:
affortable_df.dropna(subset=['avg_income'], inplace=True)

In [10]:
cost_gap_output= affortable_df.groupby("postcode").agg({'cost_gap':'mean'})


In [11]:
cost_gap_output.head()

,cost_gap
postcode,
3000,0.226212
3002,0.146635
3004,0.159437
3008,0.197724
3011,0.180597


In [12]:
print(cost_gap_output.sort_values(by=["cost_gap"],ascending=True)[:10])

          cost_gap
postcode          
3125      0.109821
3085      0.120141
3737      0.126544
3289      0.128827
3331      0.133735
3134      0.134924
3144      0.135774
3145      0.138427
3122      0.138503
3127      0.139452


In [13]:
print(cost_gap_output.sort_values(by=["cost_gap"],ascending=False)[:10])

          cost_gap
postcode          
3858      0.550120
3904      0.394804
3965      0.342909
3213      0.309285
3631      0.303043
3984      0.292961
3324      0.292790
3918      0.292274
3779      0.290680
3862      0.288469


In [14]:
cost_output= affortable_df.groupby("postcode").agg({'cost':'mean'})

In [15]:
cost_output.head()

,cost
postcode,
3000,460.439216
3002,438.421053
3004,456.125000
3008,482.244898
3011,387.425926


In [16]:
print(cost_output.sort_values(by=["cost"],ascending=True)[:20])

                cost
postcode            
3537      227.500000
3289      230.000000
3737      230.500000
3414      240.000000
3373      240.000000
3233      250.000000
3505      270.000000
3579      275.000000
3585      275.000000
3396      280.000000
3616      290.000000
3085      293.421053
3465      299.000000
3315      300.000000
3717      300.000000
3620      300.000000
3903      310.000000
3851      316.666667
3741      320.000000
3300      323.750000


In [17]:
print(cost_output.sort_values(by=["cost"],ascending=False)[:20])

                cost
postcode            
3904      557.500000
3854      550.000000
3815      550.000000
3321      550.000000
3965      540.000000
3113      540.000000
3918      535.000000
3926      530.000000
3227      522.500000
3179      510.000000
3116      510.000000
3225      503.333333
3187      501.875000
3107      497.777778
3217      496.066667
3435      495.000000
3150      493.829787
3212      491.923077
3178      490.416667
3213      490.000000


In [20]:
df_2 = pd.read_csv("../data/curated/MainHospitalList (1).csv")

In [21]:
df_2.head()

,Hospital ID,Formal Name,Other Name,Emergency Capable,Location Address,Suburb,Postcode,Access Point,Category,Agency Type
0,5488,Albert Road Clinic,Albert Road Clinic,NO,31-33 Albert Road,South Melbourne,3205,NaN,PRIVATE,Private Hospital
1,3485,Albury Wodonga Health,Albury Wodonga Health,YES,69 Vermont Street,Wodonga,3690,NaN,PUBLIC,Public Hospital
2,12990,"Albury Wodonga Health, Albury Campus","Albury Wodonga Health, Albury Campus",YES,Borella,Albury,2640,NaN,PUBLIC,Public Hospital
3,3491,Alexandra District Health,Alexandra District Health,NO,20 Cooper Street,Alexandra,3714,NaN,PUBLIC,Public Hospital
4,11519,Alfred Health,Alfred Health,NO,Commercial Road,Melbourne,3004,NaN,PUBLIC,Public Hospital


In [29]:
hospital_count = df_2.loc[:,"Postcode"].value_counts().rename_axis('postcode').reset_index(name='count')

,postcode,count
0,3199,8
1,3002,7
2,3168,6
3,3011,6
4,3101,6
...,...,...
165,3134,1
166,3478,1
167,3377,1
168,3133,1
